In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast # para la transformacion de str a array

from PIL import Image #
import re
import pandasql as sql # para trabajar con los datasets
from tqdm import tqdm # barra de progreso


from sklearn.preprocessing import MinMaxScaler # para llevar los valores de 0 a 1 no sirvio es complicado de usar


## recuperamos informacion de los arrays de la imagenes:

In [3]:
# funcion para importar dataset pick
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# funcion sql
def sql_query(query):
    q=sql.sqldf
    consulta=q(query,globals())
    return consulta

In [13]:
dataset_humano_respuestas_completo=pd.read_csv("dataset_humano_respuestas_completo.csv",index_col='indice_2')
dataset_animal_respuestas_completo=pd.read_csv("dataset_animal_respuestas_completo.csv",index_col='indice_2')
dataset_objeto_respuestas_completo=pd.read_csv("dataset_objeto_respuestas_completo.csv",index_col='indice_2')
dataset_paisaje_respuestas_completo=pd.read_csv("dataset_paisaje_respuestas_completo.csv",index_col='indice_2')

dataset_humano_cifar=unpickle('dataset_humano_cifar_250_v4_29_06_2024.pickle')
dataset_animal_cifar=unpickle('dataset_animal_cifar_250_v3_29_06_2024.pickle')
dataset_objeto_cifar=unpickle('dataset_objeto_cifar_250_v3_29_06_2024.pickle')
dataset_paisaje_cifar=unpickle('dataset_paisaje_cifar_250_v3_29_06_2024.pickle')

In [14]:
import json

def preservar_formato(x):
    return json.dumps(x.tolist())

dataset_humano_cifar['indice_2']=dataset_humano_cifar.index
dataset_humano_cifar['data_lista']=dataset_humano_cifar[b'data'].apply(preservar_formato)

dataset_animal_cifar['indice_2']=dataset_animal_cifar.index
dataset_animal_cifar['data_lista']=dataset_animal_cifar[b'data'].apply(preservar_formato)

dataset_objeto_cifar['indice_2']=dataset_objeto_cifar.index
dataset_objeto_cifar['data_lista']=dataset_objeto_cifar[b'data'].apply(preservar_formato)

dataset_paisaje_cifar['indice_2']=dataset_paisaje_cifar.index
dataset_paisaje_cifar['data_lista']=dataset_paisaje_cifar[b'data'].apply(preservar_formato)

In [15]:
query_humano_recovery_image_bytes='''
SELECT dataset_humano_respuestas_completo.*, dataset_humano_cifar.data_lista FROM dataset_humano_respuestas_completo, dataset_humano_cifar
WHERE dataset_humano_respuestas_completo.indice_2=dataset_humano_cifar.indice_2
'''

query_animal_recovery_image_bytes='''
SELECT dataset_animal_respuestas_completo.*, dataset_animal_cifar.data_lista FROM dataset_animal_respuestas_completo, dataset_animal_cifar
WHERE dataset_animal_respuestas_completo.indice_2=dataset_animal_cifar.indice_2
'''

query_objeto_recovery_image_bytes='''
SELECT dataset_objeto_respuestas_completo.*, dataset_objeto_cifar.data_lista FROM dataset_objeto_respuestas_completo, dataset_objeto_cifar
WHERE dataset_objeto_respuestas_completo.indice_2=dataset_objeto_cifar.indice_2
'''

query_paisaje_recovery_image_bytes='''
SELECT dataset_paisaje_respuestas_completo.*, dataset_paisaje_cifar.data_lista FROM dataset_paisaje_respuestas_completo, dataset_paisaje_cifar
WHERE dataset_paisaje_respuestas_completo.indice_2=dataset_paisaje_cifar.indice_2
'''

dataset_humano_respuestas_completo_bytes_reparado=sql_query(query_humano_recovery_image_bytes)

dataset_animal_respuestas_completo_bytes_reparado=sql_query(query_animal_recovery_image_bytes)

dataset_objeto_respuestas_completo_bytes_reparado=sql_query(query_objeto_recovery_image_bytes)

dataset_paisaje_respuestas_completo_bytes_reparado=sql_query(query_paisaje_recovery_image_bytes)

In [16]:
def recuperar_data_array(x): # esto es necesario para que el string que contiene la imagen se interprete de forma correcta como una estructura de datos y poder acceder a los bytes
    return np.array(json.loads(x),dtype=np.uint8) 

dataset_humano_respuestas_completo_bytes_reparado['data_lista']= dataset_humano_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_animal_respuestas_completo_bytes_reparado['data_lista']= dataset_animal_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_objeto_respuestas_completo_bytes_reparado['data_lista']= dataset_objeto_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_paisaje_respuestas_completo_bytes_reparado['data_lista']= dataset_paisaje_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)

In [17]:
### resetear index
dataset_humano_respuestas_completo_bytes_reparado.set_index(dataset_humano_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_humano_respuestas_completo_bytes_reparado = dataset_humano_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

### resetear index animal
dataset_animal_respuestas_completo_bytes_reparado.set_index(dataset_animal_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_animal_respuestas_completo_bytes_reparado = dataset_animal_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

### resetear index objeto
dataset_objeto_respuestas_completo_bytes_reparado.set_index(dataset_objeto_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_objeto_respuestas_completo_bytes_reparado = dataset_objeto_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

### resetear index paisaje
dataset_paisaje_respuestas_completo_bytes_reparado.set_index(dataset_paisaje_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_paisaje_respuestas_completo_bytes_reparado = dataset_paisaje_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

 ### Comenzamos el pre-procesamiento de las salidas de dataseet de las distintas metricas

### Dataset Filtros 1

### Texto

##### Bag

In [8]:
salida_animal_bag=pd.read_csv('Filtro_1/Texto/output_bag/salida_animal_bag_3.csv')
salida_humano_bag=pd.read_csv('Filtro_1/Texto/output_bag/salida_humano_bag_3.csv')
salida_objeto_bag=pd.read_csv('Filtro_1/Texto/output_bag/salida_objeto_bag_3.csv')
salida_paisaje_bag=pd.read_csv('Filtro_1/Texto/output_bag/salida_paisaje_bag_3.csv')

#### Cosine

In [9]:
salida_animal_cosine=pd.read_csv('Filtro_1/Texto/output_cosine/salida_animal_cosine_3.csv')
salida_humano_cosine=pd.read_csv('Filtro_1/Texto/output_cosine/salida_humano_cosine_3.csv')
salida_objeto_cosine=pd.read_csv('Filtro_1/Texto/output_cosine/salida_objeto_cosine_3.csv')
salida_paisaje_cosine=pd.read_csv('Filtro_1/Texto/output_cosine/salida_paisaje_cosine_3.csv')

#### Jaccard

In [10]:
salida_animal_jaccard=pd.read_csv('Filtro_1/Texto/output_jaccard/salida_animal_jaccard_3.csv')
salida_humano_jaccard=pd.read_csv('Filtro_1/Texto/output_jaccard/salida_humano_jaccard_3.csv')
salida_objeto_jaccard=pd.read_csv('Filtro_1/Texto/output_jaccard/salida_objeto_jaccard_3.csv')
salida_paisaje_jaccard=pd.read_csv('Filtro_1/Texto/output_jaccard/salida_paisaje_jaccard_3.csv')

#### Levenshtein

In [11]:
salida_animal_levenshtein=pd.read_csv('Filtro_1/Texto/output_levestehin/salida_prueba_animal_3.csv')
salida_humano_levenshtein=pd.read_csv('Filtro_1/Texto/output_levestehin/salida_prueba_humano_3.csv')
salida_objeto_levenshtein=pd.read_csv('Filtro_1/Texto/output_levestehin/salida_prueba_objeto_3.csv')
salida_paisaje_levenshtein=pd.read_csv('Filtro_1/Texto/output_levestehin/salida_prueba_paisaje_3.csv')

#### Overlap

In [12]:
salida_animal_overlap=pd.read_csv('Filtro_1/Texto/output_overlap/salida_animal_overlap_3.csv')
salida_humano_overlap=pd.read_csv('Filtro_1/Texto/output_overlap/salida_humano_overlap_3.csv')
salida_objeto_overlap=pd.read_csv('Filtro_1/Texto/output_overlap/salida_objeto_overlap_3.csv')
salida_paisaje_overlap=pd.read_csv('Filtro_1/Texto/output_overlap/salida_paisaje_overlap_3.csv')

### Semantica

#### Sbert

In [13]:
salida_animal_sbert=pd.read_csv('Filtro_1/semantica/sbert/df_resultado_animal_sbert_3.csv')
salida_humano_sbert=pd.read_csv('Filtro_1/semantica/sbert/df_resultado_humano_sbert_3.csv')
salida_objeto_sbert=pd.read_csv('Filtro_1/semantica/sbert/df_resultado_objeto_sbert_3.csv')
salida_paisaje_sbert=pd.read_csv('Filtro_1/semantica/sbert/df_resultado_paisaje_sbert_3.csv')

#### Web_similitary

In [14]:
salida_animal_web_similitary=pd.read_csv('Filtro_1/semantica/web_similiraty/df_resultado_animal_similarity_3.csv')
salida_humano_web_similitary=pd.read_csv('Filtro_1/semantica/web_similiraty/df_resultado_humano_similarity_3.csv')
salida_objeto_web_similitary=pd.read_csv('Filtro_1/semantica/web_similiraty/df_resultado_objeto_similarity_3.csv')
salida_paisaje_web_similitary=pd.read_csv('Filtro_1/semantica/web_similiraty/df_resultado_paisaje_similarity_3.csv')

#### Pixel_to_pixel

In [15]:
salida_animal_pixel_to_pixel=pd.read_csv('Filtro_1/pixel_to_pixel/df_resultado_animal_pixel_to_pixel_v2_3.csv')
salida_humano_pixel_to_pixel=pd.read_csv('Filtro_1/pixel_to_pixel/df_resultado_humano_pixel_to_pixel_v2_3.csv')
salida_objeto_pixel_to_pixel=pd.read_csv('Filtro_1/pixel_to_pixel/df_resultado_objeto_pixel_to_pixel_v2_3.csv')
salida_paisaje_pixel_to_pixel=pd.read_csv('Filtro_1/pixel_to_pixel/df_resultado_paisaje_pixel_to_pixel_v2_3.csv')

### Dataset Filtros 2

### Texto

#### Bag

In [16]:
salida_animal_bag_filtro_2=pd.read_csv('Filtro_2/Texto/output_bag/df_resultado_animal_bag_filtro_2_3.csv')
salida_humano_bag_filtro_2=pd.read_csv('Filtro_2/Texto/output_bag/df_resultado_humano_bag_filtro_2_3.csv')
salida_objeto_bag_filtro_2=pd.read_csv('Filtro_2/Texto/output_bag/df_resultado_objeto_bag_filtro_2_3.csv')
salida_paisaje_bag_filtro_2=pd.read_csv('Filtro_2/Texto/output_bag/df_resultado_paisaje_bag_filtro_2_3.csv')

#### Cosine

In [17]:
salida_animal_cosine_filtro_2=pd.read_csv('Filtro_2/Texto/output_cosine/df_resultado_animal_cosine_filtro_2_3.csv')
salida_humano_cosine_filtro_2=pd.read_csv('Filtro_2/Texto/output_cosine/df_resultado_humano_cosine_filtro_2_3.csv')
salida_objeto_cosine_filtro_2=pd.read_csv('Filtro_2/Texto/output_cosine/df_resultado_objeto_cosine_filtro_2_3.csv')
salida_paisaje_cosine_filtro_2=pd.read_csv('Filtro_2/Texto/output_cosine/df_resultado_paisaje_cosine_filtro_2_3.csv')

#### Jaccard

In [18]:
salida_animal_jaccard_filtro_2=pd.read_csv('Filtro_2/Texto/output_jaccard/df_resultado_animal_jaccard_filtro_2_3.csv')
salida_humano_jaccard_filtro_2=pd.read_csv('Filtro_2/Texto/output_jaccard/df_resultado_humano_jaccard_filtro_2_3.csv')
salida_objeto_jaccard_filtro_2=pd.read_csv('Filtro_2/Texto/output_jaccard/df_resultado_objeto_jaccard_filtro_2_3.csv')
salida_paisaje_jaccard_filtro_2=pd.read_csv('Filtro_2/Texto/output_jaccard/df_resultado_paisaje_jaccard_filtro_2_3.csv')

#### Levenshtein

In [19]:
salida_animal_levenshtein_filtro_2=pd.read_csv('Filtro_2/Texto/output_levestehin/df_resultado_animal_levestein_filtro_2_3.csv')
salida_humano_levenshtein_filtro_2=pd.read_csv('Filtro_2/Texto/output_levestehin/df_resultado_humano_levestein_filtro_2_3.csv')
salida_objeto_levenshtein_filtro_2=pd.read_csv('Filtro_2/Texto/output_levestehin/df_resultado_objeto_levestein_filtro_2_3.csv')
salida_paisaje_levenshtein_filtro_2=pd.read_csv('Filtro_2/Texto/output_levestehin/df_resultado_paisaje_levestein_filtro_2_3.csv')

#### Overlap

In [20]:
salida_animal_overlap_filtro_2=pd.read_csv('Filtro_2/Texto/output_overlap/df_resultado_animal_overlap_filtro_2_3.csv')
salida_humano_overlap_filtro_2=pd.read_csv('Filtro_2/Texto/output_overlap/df_resultado_humano_overlap_filtro_2_3.csv')
salida_objeto_overlap_filtro_2=pd.read_csv('Filtro_2/Texto/output_overlap/df_resultado_objeto_overlap_filtro_2_3.csv')
salida_paisaje_overlap_filtro_2=pd.read_csv('Filtro_2/Texto/output_overlap/df_resultado_paisaje_overlap_filtro_2_3.csv')


### Semantica

### Sbert

In [21]:
salida_animal_sbert_filtro_2=pd.read_csv('Filtro_2/semantica/sbert/df_resultado_animal_sbert_filtro2_3.csv') 
salida_humano_sbert_filtro_2=pd.read_csv('Filtro_2/semantica/sbert/df_resultado_humano_sbert_filtro2_3.csv')
salida_objeto_sbert_filtro_2=pd.read_csv('Filtro_2/semantica/sbert/df_resultado_objeto_sbert_filtro2_3.csv')
salida_paisaje_sbert_filtro_2=pd.read_csv('Filtro_2/semantica/sbert/df_resultado_paisaje_sbert_filtro2_3.csv')

#### Web similitary

In [22]:
salida_animal_web_similitary_filtro_2=pd.read_csv('Filtro_2/semantica/similarity/df_resultado_animal_similarity_filtro2_3.csv')
salida_humano_web_similitary_filtro_2=pd.read_csv('Filtro_2/semantica/similarity/df_resultado_humano_similarity_filtro2_3.csv')
salida_objeto_web_similitary_filtro_2=pd.read_csv('Filtro_2/semantica/similarity/df_resultado_objeto_similarity_filtro2_3.csv')
salida_paisaje_web_similitary_filtro_2=pd.read_csv('Filtro_2/semantica/similarity/df_resultado_paisaje_similarity_filtro2_3.csv')

### pixel_to_pixel

In [23]:
salida_animal_pixel_to_pixel_filtro_2=pd.read_csv('Filtro_2/pixel_to_pixel/df_resultado_animal_pixel_to_pixel_filtro_2_v2_3.csv')
salida_humano_pixel_to_pixel_filtro_2=pd.read_csv('Filtro_2/pixel_to_pixel/df_resultado_humano_pixel_to_pixel_filtro_2_v2_3.csv')
salida_objeto_pixel_to_pixel_filtro_2=pd.read_csv('Filtro_2/pixel_to_pixel/df_resultado_objeto_pixel_to_pixel_filtro_2_v2_3.csv')
salida_paisaje_pixel_to_pixel_filtro_2=pd.read_csv('Filtro_2/pixel_to_pixel/df_resultado_paisaje_pixel_to_pixel_filtro_2_v2_3.csv')

In [24]:
'''Diccionario de datasets filtro 1'''
diccionario_datasets_filtro_1 = {
    'salida_animal_bag': salida_animal_bag,
    'salida_humano_bag': salida_humano_bag,
    'salida_objeto_bag': salida_objeto_bag,
    'salida_paisaje_bag': salida_paisaje_bag,

    'salida_animal_cosine': salida_animal_cosine,
    'salida_humano_cosine': salida_humano_cosine,
    'salida_objeto_cosine': salida_objeto_cosine,
    'salida_paisaje_cosine': salida_paisaje_cosine,

    'salida_animal_jaccard': salida_animal_jaccard,
    'salida_humano_jaccard': salida_humano_jaccard,
    'salida_objeto_jaccard': salida_objeto_jaccard,
    'salida_paisaje_jaccard': salida_paisaje_jaccard,

    'salida_animal_levenshtein': salida_animal_levenshtein,
    'salida_humano_levenshtein': salida_humano_levenshtein,
    'salida_objeto_levenshtein': salida_objeto_levenshtein,
    'salida_paisaje_levenshtein': salida_paisaje_levenshtein,

    'salida_animal_overlap': salida_animal_overlap,
    'salida_humano_overlap': salida_humano_overlap,
    'salida_objeto_overlap': salida_objeto_overlap,
    'salida_paisaje_overlap': salida_paisaje_overlap,

    'salida_animal_sbert': salida_animal_sbert,
    'salida_humano_sbert': salida_humano_sbert,
    'salida_objeto_sbert': salida_objeto_sbert,
    'salida_paisaje_sbert': salida_paisaje_sbert,

    'salida_animal_web_similitary': salida_animal_web_similitary,
    'salida_humano_web_similitary': salida_humano_web_similitary,
    'salida_objeto_web_similitary': salida_objeto_web_similitary,
    'salida_paisaje_web_similitary': salida_paisaje_web_similitary,

    'salida_animal_pixel_to_pixel': salida_animal_pixel_to_pixel,
    'salida_humano_pixel_to_pixel': salida_humano_pixel_to_pixel,
    'salida_objeto_pixel_to_pixel': salida_objeto_pixel_to_pixel,
    'salida_paisaje_pixel_to_pixel': salida_paisaje_pixel_to_pixel
}


diccionario_datasets_filtro_2 = {
    'salida_animal_bag_filtro_2': salida_animal_bag_filtro_2,
    'salida_humano_bag_filtro_2': salida_humano_bag_filtro_2,
    'salida_objeto_bag_filtro_2': salida_objeto_bag_filtro_2,
    'salida_paisaje_bag_filtro_2': salida_paisaje_bag_filtro_2,

    'salida_animal_cosine_filtro_2': salida_animal_cosine_filtro_2,
    'salida_humano_cosine_filtro_2': salida_humano_cosine_filtro_2,
    'salida_objeto_cosine_filtro_2': salida_objeto_cosine_filtro_2,
    'salida_paisaje_cosine_filtro_2': salida_paisaje_cosine_filtro_2,

    'salida_animal_jaccard_filtro_2': salida_animal_jaccard_filtro_2,
    'salida_humano_jaccard_filtro_2': salida_humano_jaccard_filtro_2,
    'salida_objeto_jaccard_filtro_2': salida_objeto_jaccard_filtro_2,
    'salida_paisaje_jaccard_filtro_2': salida_paisaje_jaccard_filtro_2,

    'salida_animal_levenshtein_filtro_2': salida_animal_levenshtein_filtro_2,
    'salida_humano_levenshtein_filtro_2': salida_humano_levenshtein_filtro_2,
    'salida_objeto_levenshtein_filtro_2': salida_objeto_levenshtein_filtro_2,
    'salida_paisaje_levenshtein_filtro_2': salida_paisaje_levenshtein_filtro_2,

    'salida_animal_overlap_filtro_2': salida_animal_overlap_filtro_2,
    'salida_humano_overlap_filtro_2': salida_humano_overlap_filtro_2,
    'salida_objeto_overlap_filtro_2': salida_objeto_overlap_filtro_2,
    'salida_paisaje_overlap_filtro_2': salida_paisaje_overlap_filtro_2,

    'salida_animal_sbert_filtro_2': salida_animal_sbert_filtro_2,
    'salida_humano_sbert_filtro_2': salida_humano_sbert_filtro_2,
    'salida_objeto_sbert_filtro_2': salida_objeto_sbert_filtro_2,
    'salida_paisaje_sbert_filtro_2': salida_paisaje_sbert_filtro_2,

    'salida_animal_web_similitary_filtro_2': salida_animal_web_similitary_filtro_2,
    'salida_humano_web_similitary_filtro_2': salida_humano_web_similitary_filtro_2,
    'salida_objeto_web_similitary_filtro_2': salida_objeto_web_similitary_filtro_2,
    'salida_paisaje_web_similitary_filtro_2': salida_paisaje_web_similitary_filtro_2,

    'salida_animal_pixel_to_pixel_filtro_2': salida_animal_pixel_to_pixel_filtro_2,
    'salida_humano_pixel_to_pixel_filtro_2': salida_humano_pixel_to_pixel_filtro_2,
    'salida_objeto_pixel_to_pixel_filtro_2': salida_objeto_pixel_to_pixel_filtro_2,
    'salida_paisaje_pixel_to_pixel_filtro_2': salida_paisaje_pixel_to_pixel_filtro_2
}


# Eliminar aquellas filas 6666 que no valen para los averages

In [25]:
# Filtros 1
for nombre_dataset in diccionario_datasets_filtro_1:
    dataset_modificado=diccionario_datasets_filtro_1[nombre_dataset].loc[diccionario_datasets_filtro_1[nombre_dataset]["valor_comparacion"]!=6666]
    diccionario_datasets_filtro_1[nombre_dataset]=dataset_modificado


# Filtros 2
for nombre_dataset in diccionario_datasets_filtro_2:
    dataset_modificado=diccionario_datasets_filtro_2[nombre_dataset].loc[diccionario_datasets_filtro_2[nombre_dataset]["valor_comparacion"]!=6666]
    diccionario_datasets_filtro_2[nombre_dataset]=dataset_modificado

In [26]:
# Filtros 1
for nombre_dataset in diccionario_datasets_filtro_1:
    dataset_modificado=sql_query(f'''
    SELECT COMBINACION AS PAR_DE_INDICE, AVG(VALOR_COMPARACION) AS AVERAGE_METRICA FROM (
        SELECT *, par_1 || '-' || par_2 as combinacion FROM {nombre_dataset}
        ORDER BY combinacion)
     GROUP BY COMBINACION
''')
    diccionario_datasets_filtro_1[nombre_dataset]=dataset_modificado

# Filtros 2
for nombre_dataset in diccionario_datasets_filtro_2:
    dataset_modificado=sql_query(f'''
    SELECT COMBINACION AS PAR_DE_INDICE, AVG(VALOR_COMPARACION) AS AVERAGE_METRICA FROM (
        SELECT *, par_1 || '-' || par_2 as combinacion FROM {nombre_dataset}
        ORDER BY combinacion)
     GROUP BY COMBINACION
''')
    diccionario_datasets_filtro_2[nombre_dataset]=dataset_modificado

# chackpoint dura mucho

In [27]:
def normalizar_columna(x, dataset,columna):
    return (x - min(dataset.loc[:,columna])) / (max(dataset.loc[:,columna]) - min(dataset.loc[:,columna]))

# Filtros 1
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):
    diccionario_datasets_filtro_1[nombre_dataset].loc[:,"AVERAGE_METRICA_NORMALIZE"]=diccionario_datasets_filtro_1[nombre_dataset].loc[:,"AVERAGE_METRICA"].apply(lambda x: normalizar_columna(x,diccionario_datasets_filtro_1[nombre_dataset],"AVERAGE_METRICA"))

#Filtros 2
for nombre_dataset in tqdm(diccionario_datasets_filtro_2):
    diccionario_datasets_filtro_2[nombre_dataset].loc[:,"AVERAGE_METRICA_NORMALIZE"]=diccionario_datasets_filtro_2[nombre_dataset].loc[:,"AVERAGE_METRICA"].apply(lambda x: normalizar_columna(x,diccionario_datasets_filtro_2[nombre_dataset],"AVERAGE_METRICA"))

100%|██████████| 32/32 [45:14<00:00, 84.82s/it]  


# recomendacion guardar los datasets porque la normalizacion dura demasiado

In [28]:
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):
    diccionario_datasets_filtro_1[nombre_dataset].to_csv(f'Respaldo_procesados/filtro_1/{nombre_dataset}_procesado_for_evaluacion.csv')

for nombre_dataset in tqdm(diccionario_datasets_filtro_2):
    diccionario_datasets_filtro_2[nombre_dataset].to_csv(f'Respaldo_procesados/filtro_2/{nombre_dataset}_procesado_for_evaluacion.csv')

100%|██████████| 32/32 [00:01<00:00, 24.82it/s]


# si es necesario cargar desde aca

In [18]:
'''Diccionario de datasets filtro 1'''
diccionario_datasets_filtro_1 = {
    'salida_animal_bag': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_bag_procesado_for_evaluacion.csv'),
    'salida_humano_bag': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_bag_procesado_for_evaluacion.csv'),
    'salida_objeto_bag': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_bag_procesado_for_evaluacion.csv'),
    'salida_paisaje_bag': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_bag_procesado_for_evaluacion.csv'),

    'salida_animal_cosine': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_cosine_procesado_for_evaluacion.csv'),
    'salida_humano_cosine': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_cosine_procesado_for_evaluacion.csv'),
    'salida_objeto_cosine': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_cosine_procesado_for_evaluacion.csv'),
    'salida_paisaje_cosine': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_cosine_procesado_for_evaluacion.csv'),

    'salida_animal_jaccard': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_jaccard_procesado_for_evaluacion.csv'),
    'salida_humano_jaccard': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_jaccard_procesado_for_evaluacion.csv'),
    'salida_objeto_jaccard': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_jaccard_procesado_for_evaluacion.csv'),
    'salida_paisaje_jaccard': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_jaccard_procesado_for_evaluacion.csv'),

    'salida_animal_levenshtein': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_levenshtein_procesado_for_evaluacion.csv'),
    'salida_humano_levenshtein': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_levenshtein_procesado_for_evaluacion.csv'),
    'salida_objeto_levenshtein': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_levenshtein_procesado_for_evaluacion.csv'),
    'salida_paisaje_levenshtein': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_levenshtein_procesado_for_evaluacion.csv'),

    'salida_animal_overlap': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_overlap_procesado_for_evaluacion.csv'),
    'salida_humano_overlap': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_overlap_procesado_for_evaluacion.csv'),
    'salida_objeto_overlap': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_overlap_procesado_for_evaluacion.csv'),
    'salida_paisaje_overlap': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_overlap_procesado_for_evaluacion.csv'),

    'salida_animal_sbert': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_sbert_procesado_for_evaluacion.csv'),
    'salida_humano_sbert': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_sbert_procesado_for_evaluacion.csv'),
    'salida_objeto_sbert': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_sbert_procesado_for_evaluacion.csv'),
    'salida_paisaje_sbert': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_sbert_procesado_for_evaluacion.csv'),

    'salida_animal_web_similitary': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_web_similitary_procesado_for_evaluacion.csv'),
    'salida_humano_web_similitary': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_sbert_procesado_for_evaluacion.csv'),
    'salida_objeto_web_similitary': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_web_similitary_procesado_for_evaluacion.csv'),
    'salida_paisaje_web_similitary': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_web_similitary_procesado_for_evaluacion.csv'),

    'salida_animal_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_1/salida_animal_pixel_to_pixel_procesado_for_evaluacion.csv'),
    'salida_humano_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_1/salida_humano_pixel_to_pixel_procesado_for_evaluacion.csv'),
    'salida_objeto_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_1/salida_objeto_pixel_to_pixel_procesado_for_evaluacion.csv'),
    'salida_paisaje_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_1/salida_paisaje_pixel_to_pixel_procesado_for_evaluacion.csv')
}



'''Diccionario de datasets filtro 2'''
diccionario_datasets_filtro_2 = {
    'salida_animal_bag': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_bag_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_bag': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_bag_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_bag': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_bag_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_bag': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_bag_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_cosine': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_cosine_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_cosine': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_cosine_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_cosine': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_cosine_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_cosine': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_cosine_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_jaccard': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_jaccard_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_jaccard': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_jaccard_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_jaccard': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_jaccard_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_jaccard': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_jaccard_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_levenshtein': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_levenshtein_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_levenshtein': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_levenshtein_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_levenshtein': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_levenshtein_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_levenshtein': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_levenshtein_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_overlap': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_overlap_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_overlap': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_overlap_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_overlap': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_overlap_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_overlap': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_overlap_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_sbert': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_sbert_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_sbert': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_sbert_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_sbert': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_sbert_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_sbert': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_sbert_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_web_similitary': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_web_similitary_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_web_similitary': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_web_similitary_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_web_similitary': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_web_similitary_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_web_similitary': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_web_similitary_filtro_2_procesado_for_evaluacion.csv'),

    'salida_animal_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_2/salida_animal_pixel_to_pixel_filtro_2_procesado_for_evaluacion.csv'),
    'salida_humano_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_2/salida_humano_pixel_to_pixel_filtro_2_procesado_for_evaluacion.csv'),
    'salida_objeto_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_2/salida_objeto_pixel_to_pixel_filtro_2_procesado_for_evaluacion.csv'),
    'salida_paisaje_pixel_to_pixel': pd.read_csv('Respaldo_procesados/filtro_2/salida_paisaje_pixel_to_pixel_filtro_2_procesado_for_evaluacion.csv')
}

In [19]:
def extraer_par1(x):
    return int(re.search('^.*(?=-)',x).group())

def extraer_par2(x):
    return int(re.search('(?<=-).*',x).group())

def extrar_pares(dataset):
    dataset.loc[:,'PAR_1']=dataset.loc[:,'PAR_DE_INDICE'].apply(extraer_par1)
    dataset.loc[:,'PAR_2']=dataset.loc[:,'PAR_DE_INDICE'].apply(extraer_par2)
    return dataset

# Filtros 1
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):
    diccionario_datasets_filtro_1[nombre_dataset] =extrar_pares(diccionario_datasets_filtro_1[nombre_dataset])

# Filtros 2
for nombre_dataset in tqdm(diccionario_datasets_filtro_2):
    diccionario_datasets_filtro_2[nombre_dataset] =extrar_pares(diccionario_datasets_filtro_2[nombre_dataset])

100%|██████████| 32/32 [00:01<00:00, 26.73it/s]


In [21]:
# Filtro 1
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):
    diccionario_datasets_filtro_1[nombre_dataset]=diccionario_datasets_filtro_1[nombre_dataset].drop(['indice_main_x','indice_main_y'],axis=1).rename(columns={'data_lista_x':'data_PAR_1','data_lista_y':'data_PAR_2'})

# Filtro 2
for nombre_dataset in tqdm(diccionario_datasets_filtro_2):
    diccionario_datasets_filtro_2[nombre_dataset]=diccionario_datasets_filtro_2[nombre_dataset].drop(['indice_main_x','indice_main_y'],axis=1).rename(columns={'data_lista_x':'data_PAR_1','data_lista_y':'data_PAR_2'})

100%|██████████| 32/32 [00:00<00:00, 409.98it/s]


In [20]:
# Filtros 1
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):

    if "humano" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_humano_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_humano_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    
    elif "animal" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_animal_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_animal_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    
    elif "objeto" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_objeto_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_objeto_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    
    elif "paisaje" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_paisaje_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset], dataset_paisaje_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    


# Filtros 2
for nombre_dataset in tqdm(diccionario_datasets_filtro_2):

    if "humano" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_humano_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_humano_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    
    elif "animal" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_animal_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_animal_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    
    elif "objeto" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_objeto_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_objeto_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    
    elif "paisaje" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_paisaje_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_1', right_on='indice_main')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset], dataset_paisaje_respuestas_completo_bytes_reparado[['indice_main','data_lista','etiqueta']], left_on='PAR_2', right_on='indice_main')
    

100%|██████████| 32/32 [00:00<00:00, 185.95it/s]


### Funcion K para extraer las imagenes mas cercanas dando una referencia y cuantas k se quiere mas parecido

In [27]:
'''def funcion_k(dataset,par:int,k:int)-> pd.DataFrame:
    consulta = dataset[dataset['PAR_1'] == par]
    #### Tarea ordenar dependiendo del comportamiento de la metrica
    consulta = consulta.sort_values(by='AVERAGE_METRICA_NORMALIZE', ascending=False)
    consulta = consulta.head(k)
    return consulta
'''
def funcion_k(dataset,par:int,k:int,nombre_dataset:str)-> pd.DataFrame:
    # La metrica levenshetein funciona diferente entre el numero sea myor es mas diferente
    if "levenshtein" in nombre_dataset:
        consulta = dataset[dataset['PAR_1'] == par]
        consulta = consulta.sort_values(by='AVERAGE_METRICA_NORMALIZE', ascending=True)
        consulta = consulta.head(k)
        return consulta
    else:
        consulta = dataset[dataset['PAR_1'] == par]
        consulta = consulta.sort_values(by='AVERAGE_METRICA_NORMALIZE', ascending=False)
        consulta = consulta.head(k)
        return consulta

# Evaluacion metrica GRANCAM + ROAD

In [1]:
%pip install sewar # Para rmse deprecated

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11420 sha256=8e0eabdc07c8820876b43d4fe487aa0d3c22f4ff7df198b212e00aef3ba71c97
  Stored in directory: /home/angel/.cache/pip/wheels/87/f9/d9/6ec7e7b470df5ba0e317c6988ba7677a124ab220f5effea702
Successfully built sewar

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.9 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.


In [34]:
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 653.3 kB/s eta 0:00:0000:0100:03
Note: you may need to restart the kernel to use updated packages.


In [1]:
#!pip install grad-cam
from torchvision import models as models
import torch
import detectors
import timm
#https://huggingface.co/edadaltocg/resnet18_cifar100
#https://huggingface.co/datasets/uoft-cs/cifar100
#https://jacobgil.github.io/pytorch-gradcam-book/CAM%20Metrics%20And%20Tuning%20Tutorial.html
model = timm.create_model("resnet18_cifar100", pretrained=True)
model.eval()

/home/angel/miniconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/home/angel/miniconda3/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: '/home/angel/miniconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops9from_file4callEN3c1017basic_string_viewIcEESt8optionalIbES5_IlES5_INS2_10ScalarTypeEES5_INS2_6LayoutEES5_INS2_6DeviceEES6_'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/angel/miniconda3/lib/python3.11/site-packages/tqdm

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_

In [15]:
# base de datos etiquetas
base_etiquetas= pd.read_excel("etiquetas_cifar100.xlsx")

#funcion para sacar la etiqueta
def extraer_etiqueta(indice,dataset_parametro):
    #extraer la etiqueta de la imagen
    etiqueta= dataset_parametro[dataset_parametro["PAR_1"]==indice]["etiqueta_x"].iloc[0]
    
    #query extracion de id etiqueta
    query_extracion_id_etiqueta=f'''
    SELECT id from base_etiquetas
    WHERE TRIM(Etiqueta) = '{etiqueta}'
    '''
    return sql_query(query_extracion_id_etiqueta).iloc[0]

In [16]:
def Grandcam_ROAD(indice,modelo_grancam,dataset_parametro):

    import cv2
    import numpy as np
    
    from pytorch_grad_cam import    GradCAM, GradCAMPlusPlus
    from pytorch_grad_cam import GuidedBackpropReLUModel
    from pytorch_grad_cam.utils.image import show_cam_on_image, deprocess_image, preprocess_image

    from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
    #from torchvision.models import resnet50

    from pytorch_grad_cam.metrics.road import ROADCombined

    
    target_layers = [modelo_grancam.layer4]

    #seria llamar la funicon extraer_eqitueta y la devolucion seria el argumento de classifierOuput
    categoria= extraer_etiqueta(indice,dataset_parametro)

    targets = [ClassifierOutputTarget(categoria)]

    # procesamiento de imagen
    img= dataset_parametro[dataset_parametro["PAR_1"]==indice]["data_PAR_1"].iloc[0].reshape(3,32,32).transpose(1,2,0)
    img = cv2.resize(img, (224, 224))
    img = np.float32(img) / 255
    input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    with GradCAM(model=modelo_grancam, target_layers=target_layers) as cam:
        grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
        #cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)
    # la visualizacion creo que sobra
    #Image.fromarray(cam_image)

    # ROAD
    cam_metric = ROADCombined(percentiles=[20,40,60,80])
    scores = cam_metric(input_tensor, grayscale_cams * 0, targets, modelo_grancam)
    #print(f"Empty CAM, Combined metric avg confidence increase with ROAD accross 4 thresholds (positive is better): {scores[0]}")
    return scores[0]

In [17]:
### Sacar los indices por categoria

def extraer_indices():

    cont_humano=0
    cont_animal=0
    cont_objeto=0
    cont_paisaje=0

    for nombre_dataset in tqdm(diccionario_datasets_filtro_1):
        
        if ('humano' in nombre_dataset) & (cont_humano<1):
            cont_humano+=1
            indices_humano=diccionario_datasets_filtro_1[nombre_dataset].PAR_1.drop_duplicates()
        elif ('animal' in nombre_dataset) & (cont_animal<1) :
            cont_animal+=1
            indices_animal=diccionario_datasets_filtro_1[nombre_dataset].PAR_1.drop_duplicates()

        elif ('objeto' in nombre_dataset) & (cont_objeto<1) :
            cont_objeto+=1
            indices_objeto=diccionario_datasets_filtro_1[nombre_dataset].PAR_1.drop_duplicates()

        elif ('paisaje' in nombre_dataset) & (cont_paisaje<1) :
            cont_paisaje+=1
            indices_paisaje=diccionario_datasets_filtro_1[nombre_dataset].PAR_1.drop_duplicates()
        else:
            break

    return indices_humano,indices_animal,indices_objeto,indices_paisaje

def extraer_data():

    diccionario_datasets_filtro_2[nombre_dataset]
    

indices_humano,indices_animal,indices_objeto,indices_paisaje= extraer_indices()

    
### luego ccon los indices sacar los ybtes
## luego calcular el road
# y luego asignar los road a todas los datasets por categoria

 12%|█▎        | 4/32 [00:00<00:00, 4473.92it/s]


In [18]:
def extaer_bytes(indices_humano,indices_animal,indices_objeto,indices_paisaje):
    cont_humano=0
    cont_animal=0
    cont_objeto=0
    cont_paisaje=0

    bytes_humano=[]
    bytes_animal=[]
    bytes_objeto=[]
    bytes_paisaje=[]
    
    for nombre_dataset in tqdm(diccionario_datasets_filtro_1): # da igual si es filtro 1 o 2 o la metrica los baytes son los mismo.
                
        if ('humano' in nombre_dataset) & (cont_humano<1):
            cont_humano+=1
            for x in indices_humano:
                valor=diccionario_datasets_filtro_1["salida_humano_bag"].loc[diccionario_datasets_filtro_1["salida_humano_bag"]["PAR_1"]==x,"data_PAR_1"].iloc[0]
                bytes_humano.append(valor)

        elif ('animal' in nombre_dataset) & (cont_animal<1) :
            cont_animal+=1
            for x in indices_animal:
                valor=diccionario_datasets_filtro_1["salida_animal_bag"].loc[diccionario_datasets_filtro_1["salida_animal_bag"]["PAR_1"]==x,"data_PAR_1"].iloc[0]
                bytes_animal.append(valor)

        elif ('objeto' in nombre_dataset) & (cont_objeto<1) :
            cont_objeto+=1
            for x in indices_objeto:
                valor=diccionario_datasets_filtro_1["salida_objeto_bag"].loc[diccionario_datasets_filtro_1["salida_objeto_bag"]["PAR_1"]==x,"data_PAR_1"].iloc[0]
                bytes_objeto.append(valor)

        elif ('paisaje' in nombre_dataset) & (cont_paisaje<1) :
            cont_paisaje+=1
            for x in indices_paisaje:
                valor=diccionario_datasets_filtro_1["salida_paisaje_bag"].loc[diccionario_datasets_filtro_1["salida_paisaje_bag"]["PAR_1"]==x,"data_PAR_1"].iloc[0]
                bytes_paisaje.append(valor)

        else:
            break

    return bytes_humano,bytes_animal,bytes_objeto,bytes_paisaje

bytes_humano,bytes_animal,bytes_objeto,bytes_paisaje=extaer_bytes(indices_humano,indices_animal,indices_objeto,indices_paisaje)



 12%|█▎        | 4/32 [00:00<00:00, 32.65it/s]


In [19]:
# unir listas a ver

Lista_indice_bytes_humano=list(zip(indices_humano,bytes_humano))

Lista_indice_bytes_animal=list(zip(indices_animal,bytes_animal))

Lista_indice_bytes_objeto=list(zip(indices_objeto,bytes_objeto))

Lista_indice_bytes_paisaje=list(zip(indices_paisaje,bytes_paisaje))

In [20]:
def Apply_Grancam_ROAD(model):
    ROAD_humano=[]
    ROAD_animal=[]
    ROAD_objeto=[]
    ROAD_paisaje=[]
    
    for humano,animal,objeto,paisaje in tqdm(zip(Lista_indice_bytes_humano,Lista_indice_bytes_animal,Lista_indice_bytes_objeto,Lista_indice_bytes_paisaje)):

        valor_humano=Grandcam_ROAD(humano[0],model,diccionario_datasets_filtro_1["salida_humano_bag"])
        ROAD_humano.append(valor_humano)

        valor_animal=Grandcam_ROAD(animal[0],model,diccionario_datasets_filtro_1["salida_animal_bag"])
        ROAD_animal.append(valor_animal)

        valor_objeto=Grandcam_ROAD(objeto[0],model,diccionario_datasets_filtro_1["salida_objeto_bag"])
        ROAD_objeto.append(valor_objeto)

        valor_paisaje=Grandcam_ROAD(paisaje[0],model,diccionario_datasets_filtro_1["salida_paisaje_bag"])
        ROAD_paisaje.append(valor_paisaje)
        
    return ROAD_humano, ROAD_animal, ROAD_objeto, ROAD_paisaje

ROAD_humano, ROAD_animal, ROAD_objeto, ROAD_paisaje=Apply_Grancam_ROAD(model)

0it [00:00, ?it/s]/home/angel/miniconda3/lib/python3.11/site-packages/pytorch_grad_cam/utils/model_targets.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return model_output[self.category]
/home/angel/miniconda3/lib/python3.11/site-packages/pytorch_grad_cam/utils/model_targets.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return model_output[self.category]
/home/angel/miniconda3/lib/python3.11/site-packages/pytorch_grad_cam/utils/model_targets.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent w

In [21]:
Lista_indice_bytes_ROAD_humano=list(zip(indices_humano,bytes_humano, ROAD_humano))

Lista_indice_bytes_ROAD_animal=list(zip(indices_animal,bytes_animal, ROAD_animal))

Lista_indice_bytes_ROAD_objeto=list(zip(indices_objeto,bytes_objeto, ROAD_objeto))

Lista_indice_bytes_ROAD_paisaje=list(zip(indices_paisaje,bytes_paisaje, ROAD_paisaje))

del Lista_indice_bytes_humano, Lista_indice_bytes_animal, Lista_indice_bytes_objeto, Lista_indice_bytes_paisaje

In [22]:
#humano
dataset_lista_indice_bytes_ROAD_humano=pd.DataFrame(columns=["indice","bytes_humano","ROAD_humano"],data=Lista_indice_bytes_ROAD_humano)
dataset_lista_indice_bytes_ROAD_humano.to_pickle("dataset_lista_indice_bytes_ROAD_humano.pkl")

#animal
dataset_lista_indice_bytes_ROAD_animal=pd.DataFrame(columns=["indice","bytes_animal","ROAD_animal"],data=Lista_indice_bytes_ROAD_animal)
dataset_lista_indice_bytes_ROAD_animal.to_pickle("dataset_lista_indice_bytes_ROAD_animal.pkl")


#objeto
dataset_lista_indice_bytes_ROAD_objeto=pd.DataFrame(columns=["indice","bytes_objeto","ROAD_objeto"],data=Lista_indice_bytes_ROAD_objeto)
dataset_lista_indice_bytes_ROAD_objeto.to_pickle("dataset_lista_indice_bytes_ROAD_objeto.pkl")


#paisaje
dataset_lista_indice_bytes_ROAD_paisaje=pd.DataFrame(columns=["indice","bytes_paisaje","ROAD_paisaje"],data=Lista_indice_bytes_ROAD_paisaje)
dataset_lista_indice_bytes_ROAD_paisaje.to_pickle("dataset_lista_indice_bytes_ROAD_paisaje.pkl")



## Checkpoint cargando valores ROAD

In [22]:
dataset_lista_indice_bytes_ROAD_humano=unpickle("dataset_lista_indice_bytes_ROAD_humano.pkl")
dataset_lista_indice_bytes_ROAD_humano=dataset_lista_indice_bytes_ROAD_humano.rename(columns={"ROAD_humano":"ROAD","bytes_humano":"bytes"})

dataset_lista_indice_bytes_ROAD_animal=unpickle("dataset_lista_indice_bytes_ROAD_animal.pkl")
dataset_lista_indice_bytes_ROAD_animal=dataset_lista_indice_bytes_ROAD_animal.rename(columns={"ROAD_animal":"ROAD","bytes_animal":"bytes"})

dataset_lista_indice_bytes_ROAD_objeto=unpickle("dataset_lista_indice_bytes_ROAD_objeto.pkl")
dataset_lista_indice_bytes_ROAD_objeto=dataset_lista_indice_bytes_ROAD_objeto.rename(columns={"ROAD_objeto":"ROAD","bytes_objeto":"bytes"})

dataset_lista_indice_bytes_ROAD_paisaje=unpickle("dataset_lista_indice_bytes_ROAD_paisaje.pkl")
dataset_lista_indice_bytes_ROAD_paisaje=dataset_lista_indice_bytes_ROAD_paisaje.rename(columns={"ROAD_paisaje":"ROAD","bytes_paisaje":"bytes"})

## Normalizacion de valores ROAD

In [23]:
#Por el rango de valores de la metrica ROAD se normaliza estos

def normalizar_columna_for_ROAD(x): 
# los valores los saque manualmente de los ROADs de las distintas categorias. al ser todas provenietes de magnitudes normalizadas del 0 al 1 seria un solo rango de valores ROAD
    return (x - (-0.02568936)) / (0.042063 - (-0.02568936)) 

dataset_lista_indice_bytes_ROAD_humano.loc[:,"ROAD"]=dataset_lista_indice_bytes_ROAD_humano.loc[:,"ROAD"].apply(normalizar_columna_for_ROAD)


dataset_lista_indice_bytes_ROAD_animal.loc[:,"ROAD"]=dataset_lista_indice_bytes_ROAD_animal.loc[:,"ROAD"].apply(normalizar_columna_for_ROAD)


dataset_lista_indice_bytes_ROAD_objeto.loc[:,"ROAD"]=dataset_lista_indice_bytes_ROAD_objeto.loc[:,"ROAD"].apply(normalizar_columna_for_ROAD)


dataset_lista_indice_bytes_ROAD_paisaje.loc[:,"ROAD"]=dataset_lista_indice_bytes_ROAD_paisaje.loc[:,"ROAD"].apply(normalizar_columna_for_ROAD)


# Realizando los inner para añadir el valor de los ROAD

In [24]:
# Filtros 1
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):

    if "humano" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_humano, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_humano, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset]=diccionario_datasets_filtro_1[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

    elif "animal" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_animal, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_animal, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset]=diccionario_datasets_filtro_1[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

    elif "objeto" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_objeto, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_objeto, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset]=diccionario_datasets_filtro_1[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

    elif "paisaje" in nombre_dataset:
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_paisaje, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset] = pd.merge(diccionario_datasets_filtro_1[nombre_dataset],dataset_lista_indice_bytes_ROAD_paisaje, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_1[nombre_dataset]=diccionario_datasets_filtro_1[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

# Filtros 2
for nombre_dataset in tqdm(diccionario_datasets_filtro_2):
    
    if "humano" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_humano, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_humano, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset]=diccionario_datasets_filtro_2[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

    elif "animal" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_animal, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_animal, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset]=diccionario_datasets_filtro_2[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

    elif "objeto" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_objeto, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_objeto, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset]=diccionario_datasets_filtro_2[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})


    elif "paisaje" in nombre_dataset:
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_paisaje, left_on='PAR_1', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset] = pd.merge(diccionario_datasets_filtro_2[nombre_dataset],dataset_lista_indice_bytes_ROAD_paisaje, left_on='PAR_2', right_on='indice')
        diccionario_datasets_filtro_2[nombre_dataset]=diccionario_datasets_filtro_2[nombre_dataset].drop(["etiqueta_y","indice_x","indice_y","bytes_x","bytes_y"],axis=1).rename(columns={"etiqueta_x":"etiqueta","ROAD_x":"ROAD_PAR_1","ROAD_y":"ROAD_PAR_2"})

    

100%|██████████| 32/32 [00:00<00:00, 121.71it/s]


### Funciones para evaluacion con la metrica road

In [25]:
def calcular_promedio_de_rmse_k1(x):
    return (x[0][1])/1

def calcular_promedio_de_rmse_k2(x):
    return (x[0][1]+ x[1][1])/2

def calcular_promedio_de_rmse_k3(x):
    return (x[0][1]+ x[1][1]+ x[2][1])/3

def calcular_promedio_de_rmse_k4(x):
    return (x[0][1]+ x[1][1]+ x[2][1]+ x[3][1])/4

def transformar_cadena(x):
    return eval(x)

''''
def rmse_manual(image_ref, image_comp):
    from skimage.metrics import mean_squared_error
    return mean_squared_error(image_ref,image_comp)
'''
     
def evaluacion_ROAD(dataset,k,nombre_dataset):
    from sewar.full_ref import rmse #type:ignore
    
    for indice in dataset.PAR_1.drop_duplicates():

        #img_ref=dataset['data_PAR_1'].loc[dataset['PAR_1']==indice].iloc[0].reshape(3,32,32).transpose(1,2,0)

        mejores_valores = funcion_k(dataset,indice,k,nombre_dataset)[['PAR_2','data_PAR_2','ROAD_PAR_2']] # antes renombrar las columnas

        #evaluaciones=[rmse(img_ref,mejores_valores['data_PAR_2'].iloc[x].reshape(3,32,32).transpose(1,2,0)) for x in range(0,len(mejores_valores))]

        evaluaciones= mejores_valores["ROAD_PAR_2"]

        if k==1:
            valor= f'[({mejores_valores.iloc[0,0]},{evaluaciones.iloc[0][0]})]'
            dataset.loc[dataset['PAR_1'] == indice, f'Evaluacion_k_igual{k}'] = valor

        elif k==2:
            valor= f'[({mejores_valores.iloc[0,0]},{evaluaciones.iloc[0][0]}),({mejores_valores.iloc[1,0]},{evaluaciones.iloc[1][0]})]'
            dataset.loc[dataset['PAR_1'] == indice, f'Evaluacion_k_igual{k}'] = valor

        elif k==3:
            valor= f'[({mejores_valores.iloc[0,0]},{evaluaciones.iloc[0][0]}),({mejores_valores.iloc[1,0]},{evaluaciones.iloc[1][0]}),({mejores_valores.iloc[2,0]},{evaluaciones.iloc[2][0]})]'
            dataset.loc[dataset['PAR_1'] == indice, f'Evaluacion_k_igual{k}'] = valor

    
    if k==1:
        dataset.loc[:,f'Evaluacion_k_igual{k}']=dataset.loc[:,f'Evaluacion_k_igual{k}'].apply(transformar_cadena)
        dataset.loc[:,f'promedio_{k}']=dataset.loc[:,f'Evaluacion_k_igual{k}'].apply(calcular_promedio_de_rmse_k1)

    elif k==2:
        dataset.loc[:,f'Evaluacion_k_igual{k}']=dataset.loc[:,f'Evaluacion_k_igual{k}'].apply(transformar_cadena)
        dataset.loc[:,f'promedio_{k}']=dataset.loc[:,f'Evaluacion_k_igual{k}'].apply(calcular_promedio_de_rmse_k2)

    elif k==3:
        dataset.loc[:,f'Evaluacion_k_igual{k}']=dataset.loc[:,f'Evaluacion_k_igual{k}'].apply(transformar_cadena)
        dataset.loc[:,f'promedio_{k}']=dataset.loc[:,f'Evaluacion_k_igual{k}'].apply(calcular_promedio_de_rmse_k3)
    
    return dataset

# Evaluacion

In [28]:
#Aqui saco los road mas cercanos (k1, k2, k3)

# Filtro 1
for nombre_dataset in tqdm(diccionario_datasets_filtro_1):
    diccionario_datasets_filtro_1[nombre_dataset]=evaluacion_ROAD(diccionario_datasets_filtro_1[nombre_dataset],1,nombre_dataset)
    diccionario_datasets_filtro_1[nombre_dataset]=evaluacion_ROAD(diccionario_datasets_filtro_1[nombre_dataset],2,nombre_dataset)
    diccionario_datasets_filtro_1[nombre_dataset]=evaluacion_ROAD(diccionario_datasets_filtro_1[nombre_dataset],3,nombre_dataset)

# Filtro 2
for nombre_dataset in tqdm(diccionario_datasets_filtro_2):
    diccionario_datasets_filtro_2[nombre_dataset]=evaluacion_ROAD(diccionario_datasets_filtro_2[nombre_dataset],1,nombre_dataset)
    diccionario_datasets_filtro_2[nombre_dataset]=evaluacion_ROAD(diccionario_datasets_filtro_2[nombre_dataset],2,nombre_dataset)
    diccionario_datasets_filtro_2[nombre_dataset]=evaluacion_ROAD(diccionario_datasets_filtro_2[nombre_dataset],3,nombre_dataset)

100%|██████████| 32/32 [00:41<00:00,  1.29s/it]


## Agrupameinto de dataset para porcentaje de mejoras

In [29]:
# Filtro 1
    # bag
dataset_bag_filtro_1_evaluaciones=pd.concat([diccionario_datasets_filtro_1["salida_animal_bag"],
                                            diccionario_datasets_filtro_1["salida_humano_bag"],
                                            diccionario_datasets_filtro_1["salida_objeto_bag"],
                                            diccionario_datasets_filtro_1["salida_paisaje_bag"]])
    
    #cosine
dataset_cosine_filtro_1_evaluaciones=pd.concat([
                                                diccionario_datasets_filtro_1["salida_animal_cosine"],
                                                diccionario_datasets_filtro_1["salida_humano_cosine"],
                                                diccionario_datasets_filtro_1["salida_objeto_cosine"],
                                                diccionario_datasets_filtro_1["salida_paisaje_cosine"]])

    
    #jaccard
dataset_jaccard_filtro_1_evaluaciones=pd.concat([
                                                diccionario_datasets_filtro_1["salida_animal_jaccard"],
                                                diccionario_datasets_filtro_1["salida_humano_jaccard"],
                                                diccionario_datasets_filtro_1["salida_objeto_jaccard"],
                                                diccionario_datasets_filtro_1["salida_paisaje_jaccard"]])

    #levenshtein
dataset_levenshtein_filtro_1_evaluaciones=pd.concat([diccionario_datasets_filtro_1["salida_animal_levenshtein"],
                                                        diccionario_datasets_filtro_1["salida_humano_levenshtein"],
                                                        diccionario_datasets_filtro_1["salida_objeto_levenshtein"],
                                                        diccionario_datasets_filtro_1["salida_paisaje_levenshtein"]])

    #overlap
dataset_overlap_filtro_1_evaluaciones=pd.concat([diccionario_datasets_filtro_1["salida_animal_overlap"],
                                                diccionario_datasets_filtro_1["salida_humano_overlap"],
                                                diccionario_datasets_filtro_1["salida_objeto_overlap"],
                                                diccionario_datasets_filtro_1["salida_paisaje_overlap"]])

    #sbert
dataset_sbert_filtro_1_evaluaciones=pd.concat([diccionario_datasets_filtro_1["salida_animal_sbert"],
                                                diccionario_datasets_filtro_1["salida_humano_sbert"],
                                                diccionario_datasets_filtro_1["salida_objeto_sbert"],
                                                diccionario_datasets_filtro_1["salida_paisaje_sbert"]])

    #web similitary
dataset_web_similitary_filtro_1_evaluaciones=pd.concat([diccionario_datasets_filtro_1["salida_animal_web_similitary"],
                                                        diccionario_datasets_filtro_1["salida_humano_web_similitary"],
                                                        diccionario_datasets_filtro_1["salida_objeto_web_similitary"],
                                                        diccionario_datasets_filtro_1["salida_paisaje_web_similitary"]])

    #pixel_to_pixel
dataset_pixel_to_pixel_filtro_1_evaluaciones=pd.concat([
                                                        diccionario_datasets_filtro_1["salida_animal_pixel_to_pixel"],
                                                        diccionario_datasets_filtro_1["salida_humano_pixel_to_pixel"],
                                                        diccionario_datasets_filtro_1["salida_objeto_pixel_to_pixel"],
                                                        diccionario_datasets_filtro_1["salida_paisaje_pixel_to_pixel"]])

## Exportando evaluaciones en bruto

In [31]:
# Filtro 1
    # bag
dataset_bag_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_bag_filtro_1_evaluaciones.xlsx")
    
    #cosine
dataset_cosine_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_cosine_filtro_1_evaluaciones.xlsx")

    
    #jaccard
dataset_jaccard_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_jaccard_filtro_1_evaluaciones.xlsx")

    #levenshtein
dataset_levenshtein_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_levenshtein_filtro_1_evaluaciones.xlsx")

    #overlap
dataset_overlap_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_overlap_filtro_1_evaluaciones.xlsx")

    #sbert
dataset_sbert_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_sbert_filtro_1_evaluaciones.xlsx")

    #web similitary
dataset_web_similitary_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_web_similitary_filtro_1_evaluaciones.xlsx")

    #pixel_to_pixel
dataset_pixel_to_pixel_filtro_1_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_pixel_to_pixel_filtro_1_evaluaciones.xlsx")

In [30]:
# Filtro 2
    # bag
dataset_bag_filtro_2_evaluaciones=pd.concat([diccionario_datasets_filtro_2["salida_animal_bag"],
                                            diccionario_datasets_filtro_2["salida_humano_bag"],
                                            diccionario_datasets_filtro_2["salida_objeto_bag"],
                                            diccionario_datasets_filtro_2["salida_paisaje_bag"]])
    
    #cosine
dataset_cosine_filtro_2_evaluaciones=pd.concat([
                                                diccionario_datasets_filtro_2["salida_animal_cosine"],
                                                diccionario_datasets_filtro_2["salida_humano_cosine"],
                                                diccionario_datasets_filtro_2["salida_objeto_cosine"],
                                                diccionario_datasets_filtro_2["salida_paisaje_cosine"]])

    
    #jaccard
dataset_jaccard_filtro_2_evaluaciones=pd.concat([
                                                diccionario_datasets_filtro_2["salida_animal_jaccard"],
                                                diccionario_datasets_filtro_2["salida_humano_jaccard"],
                                                diccionario_datasets_filtro_2["salida_objeto_jaccard"],
                                                diccionario_datasets_filtro_2["salida_paisaje_jaccard"]])

    #levenshtein
dataset_levenshtein_filtro_2_evaluaciones=pd.concat([diccionario_datasets_filtro_2["salida_animal_levenshtein"],
                                                        diccionario_datasets_filtro_2["salida_humano_levenshtein"],
                                                        diccionario_datasets_filtro_2["salida_objeto_levenshtein"],
                                                        diccionario_datasets_filtro_2["salida_paisaje_levenshtein"]])

    #overlap
dataset_overlap_filtro_2_evaluaciones=pd.concat(
                                                [diccionario_datasets_filtro_2["salida_animal_overlap"],
                                                diccionario_datasets_filtro_2["salida_humano_overlap"],
                                                diccionario_datasets_filtro_2["salida_objeto_overlap"],
                                                diccionario_datasets_filtro_2["salida_paisaje_overlap"]])



    #sbert
dataset_sbert_filtro_2_evaluaciones=pd.concat([diccionario_datasets_filtro_2["salida_animal_sbert"],
                                                diccionario_datasets_filtro_2["salida_humano_sbert"],
                                                diccionario_datasets_filtro_2["salida_objeto_sbert"],
                                                diccionario_datasets_filtro_2["salida_paisaje_sbert"]])

    #web similitary
dataset_web_similitary_filtro_2_evaluaciones=pd.concat([diccionario_datasets_filtro_2["salida_animal_web_similitary"],
                                                        diccionario_datasets_filtro_2["salida_humano_web_similitary"],
                                                        diccionario_datasets_filtro_2["salida_objeto_web_similitary"],
                                                        diccionario_datasets_filtro_2["salida_paisaje_web_similitary"]])

    #pixel_to_pixel
dataset_pixel_to_pixel_filtro_2_evaluaciones=pd.concat([
                                                        diccionario_datasets_filtro_2["salida_animal_pixel_to_pixel"],
                                                        diccionario_datasets_filtro_2["salida_humano_pixel_to_pixel"],
                                                        diccionario_datasets_filtro_2["salida_objeto_pixel_to_pixel"],
                                                        diccionario_datasets_filtro_2["salida_paisaje_pixel_to_pixel"]])

## Exportando evaluaciones en bruto

In [32]:
# Filtro 2
    # bag
dataset_bag_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_bag_filtro_2_evaluaciones.xlsx")
    
    #cosine
dataset_cosine_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_cosine_filtro_2_evaluaciones.xlsx")

    
    #jaccard
dataset_jaccard_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_jaccard_filtro_2_evaluaciones.xlsx")

    #levenshtein
dataset_levenshtein_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_levenshtein_filtro_2_evaluaciones.xlsx")

    #overlap
dataset_overlap_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_overlap_filtro_2_evaluaciones.xlsx")

    #sbert
dataset_sbert_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_sbert_filtro_2_evaluaciones.xlsx")

    #web similitary
dataset_web_similitary_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_web_similitary_filtro_2_evaluaciones.xlsx")

    #pixel_to_pixel
dataset_pixel_to_pixel_filtro_2_evaluaciones.to_excel("Evaluaciones_en_bruto/ROAD/dataset_pixel_to_pixel_filtro_2_evaluaciones.xlsx")

## Tablas comparativas Evaluaciones (No porcentaje de mejoras)

In [50]:
#Valores para K1
Promedio_evaluacion_bag_K1_filtro_1=dataset_bag_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_cosine_K1_filtro_1= dataset_cosine_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_jaccard_K1_filtro_1= dataset_jaccard_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_levestehinh_K1_filtro_1= dataset_levenshtein_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_overlap_K1_filtro_1= dataset_overlap_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_sbert_K1_filtro_1= dataset_sbert_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_web_similitary_K1_filtro_1= dataset_web_similitary_filtro_1_evaluaciones.promedio_1.mean()
Promedio_evaluacion_pixel_K1_filtro_1= dataset_pixel_to_pixel_filtro_1_evaluaciones.promedio_1.mean()

Promedio_evaluacion_bag_K1_filtro_2=dataset_bag_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_cosine_K1_filtro_2= dataset_cosine_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_jaccard_K1_filtro_2= dataset_jaccard_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_levestehinh_K1_filtro_2= dataset_levenshtein_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_overlap_K1_filtro_2= dataset_overlap_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_sbert_K1_filtro_2= dataset_sbert_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_web_similitary_K1_filtro_2= dataset_web_similitary_filtro_2_evaluaciones.promedio_1.mean()
Promedio_evaluacion_pixel_K1_filtro_2= dataset_pixel_to_pixel_filtro_2_evaluaciones.promedio_1.mean()



Valores_k1=[

    Promedio_evaluacion_bag_K1_filtro_1,
    Promedio_evaluacion_cosine_K1_filtro_1,
    Promedio_evaluacion_jaccard_K1_filtro_1,
    Promedio_evaluacion_levestehinh_K1_filtro_1,
    Promedio_evaluacion_overlap_K1_filtro_1,
    Promedio_evaluacion_sbert_K1_filtro_1,
    Promedio_evaluacion_web_similitary_K1_filtro_1,
    Promedio_evaluacion_pixel_K1_filtro_1,

    Promedio_evaluacion_bag_K1_filtro_2,
    Promedio_evaluacion_cosine_K1_filtro_2,
    Promedio_evaluacion_jaccard_K1_filtro_2,
    Promedio_evaluacion_levestehinh_K1_filtro_2,
    Promedio_evaluacion_overlap_K1_filtro_2,
    Promedio_evaluacion_sbert_K1_filtro_2,
    Promedio_evaluacion_web_similitary_K1_filtro_2,
    Promedio_evaluacion_pixel_K1_filtro_2
]



#Valores para k2
Promedio_evaluacion_bag_k2_filtro_1=dataset_bag_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_cosine_k2_filtro_1= dataset_cosine_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_jaccard_k2_filtro_1= dataset_jaccard_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_levestehinh_k2_filtro_1= dataset_levenshtein_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_overlap_k2_filtro_1= dataset_overlap_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_sbert_k2_filtro_1= dataset_sbert_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_web_similitary_k2_filtro_1= dataset_web_similitary_filtro_1_evaluaciones.promedio_2.mean()
Promedio_evaluacion_pixel_k2_filtro_1= dataset_pixel_to_pixel_filtro_1_evaluaciones.promedio_2.mean()

Promedio_evaluacion_bag_k2_filtro_2=dataset_bag_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_cosine_k2_filtro_2= dataset_cosine_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_jaccard_k2_filtro_2= dataset_jaccard_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_levestehinh_k2_filtro_2= dataset_levenshtein_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_overlap_k2_filtro_2= dataset_overlap_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_sbert_k2_filtro_2= dataset_sbert_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_web_similitary_k2_filtro_2= dataset_web_similitary_filtro_2_evaluaciones.promedio_2.mean()
Promedio_evaluacion_pixel_k2_filtro_2= dataset_pixel_to_pixel_filtro_2_evaluaciones.promedio_2.mean()

Valores_k2=[

    Promedio_evaluacion_bag_k2_filtro_1,
    Promedio_evaluacion_cosine_k2_filtro_1,
    Promedio_evaluacion_jaccard_k2_filtro_1,
    Promedio_evaluacion_levestehinh_k2_filtro_1,
    Promedio_evaluacion_overlap_k2_filtro_1,
    Promedio_evaluacion_sbert_k2_filtro_1,
    Promedio_evaluacion_web_similitary_k2_filtro_1,
    Promedio_evaluacion_pixel_k2_filtro_1,

    Promedio_evaluacion_bag_k2_filtro_2,
    Promedio_evaluacion_cosine_k2_filtro_2,
    Promedio_evaluacion_jaccard_k2_filtro_2,
    Promedio_evaluacion_levestehinh_k2_filtro_2,
    Promedio_evaluacion_overlap_k2_filtro_2,
    Promedio_evaluacion_sbert_k2_filtro_2,
    Promedio_evaluacion_web_similitary_k2_filtro_2,
    Promedio_evaluacion_pixel_k2_filtro_2
]


#Valores para k3
Promedio_evaluacion_bag_k3_filtro_1=dataset_bag_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_cosine_k3_filtro_1= dataset_cosine_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_jaccard_k3_filtro_1= dataset_jaccard_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_levestehinh_k3_filtro_1= dataset_levenshtein_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_overlap_k3_filtro_1= dataset_overlap_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_sbert_k3_filtro_1= dataset_sbert_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_web_similitary_k3_filtro_1= dataset_web_similitary_filtro_1_evaluaciones.promedio_3.mean()
Promedio_evaluacion_pixel_k3_filtro_1= dataset_pixel_to_pixel_filtro_1_evaluaciones.promedio_3.mean()

Promedio_evaluacion_bag_k3_filtro_2=dataset_bag_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_cosine_k3_filtro_2= dataset_cosine_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_jaccard_k3_filtro_2= dataset_jaccard_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_levestehinh_k3_filtro_2= dataset_levenshtein_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_overlap_k3_filtro_2= dataset_overlap_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_sbert_k3_filtro_2= dataset_sbert_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_web_similitary_k3_filtro_2= dataset_web_similitary_filtro_2_evaluaciones.promedio_3.mean()
Promedio_evaluacion_pixel_k3_filtro_2= dataset_pixel_to_pixel_filtro_2_evaluaciones.promedio_3.mean()


Valores_k3=[

    Promedio_evaluacion_bag_k3_filtro_1,
    Promedio_evaluacion_cosine_k3_filtro_1,
    Promedio_evaluacion_jaccard_k3_filtro_1,
    Promedio_evaluacion_levestehinh_k3_filtro_1,
    Promedio_evaluacion_overlap_k3_filtro_1,
    Promedio_evaluacion_sbert_k3_filtro_1,
    Promedio_evaluacion_web_similitary_k3_filtro_1,
    Promedio_evaluacion_pixel_k3_filtro_1,

    Promedio_evaluacion_bag_k3_filtro_2,
    Promedio_evaluacion_cosine_k3_filtro_2,
    Promedio_evaluacion_jaccard_k3_filtro_2,
    Promedio_evaluacion_levestehinh_k3_filtro_2,
    Promedio_evaluacion_overlap_k3_filtro_2,
    Promedio_evaluacion_sbert_k3_filtro_2,
    Promedio_evaluacion_web_similitary_k3_filtro_2,
    Promedio_evaluacion_pixel_k3_filtro_2
]

In [51]:
Tabla_comparativa_evaluaciones= {
    'Metrica':["Bag", "Cosine", "jaccard", "levestehin","overlap", "SBERT", "Web_similitary","pixel_to_pixel", "Bag", "Cosine", "jaccard", "levestehin","overlap", "SBERT", "Web_similitary","pixel_to_pixel"],
    'Filtro':["1","1","1","1","1","1","1","1","2","2","2","2","2","2","2","2"],
    'K1':Valores_k1,
    'K2':Valores_k2,
    'k3':Valores_k3
}

Tabla_comparativa_evaluaciones=pd.DataFrame(Tabla_comparativa_evaluaciones)

In [52]:
Tabla_comparativa_evaluaciones

,Metrica,Filtro,K1,K2,k3
0,Bag,1,0.368628,0.371105,0.370663
1,Cosine,1,0.376800,0.375143,0.374446
2,jaccard,1,0.375884,0.375495,0.373901
3,levestehin,1,0.379212,0.380194,0.378501
4,overlap,1,0.379667,0.379820,0.380576
5,SBERT,1,0.376689,0.378041,0.377200
6,Web_similitary,1,0.378158,0.377339,0.379405
7,pixel_to_pixel,1,0.362278,0.368834,0.371884
8,Bag,2,0.352971,0.357842,0.360614
9,Cosine,2,0.369328,0.369213,0.369659


In [53]:
Tabla_comparativa_evaluaciones.to_excel("Tabla_comparativa_evaluaciones_ROAD.xlsx")

## Tratamiento y funciones para porcentaje de mejoras

In [65]:
diccionario_dataset_filtro_1_para_porcentajes_de_mejoras = {
    "dataset_bag_filtro_1_evaluaciones": dataset_bag_filtro_1_evaluaciones,
    "dataset_cosine_filtro_1_evaluaciones": dataset_cosine_filtro_1_evaluaciones,
    "dataset_jaccard_filtro_1_evaluaciones": dataset_jaccard_filtro_1_evaluaciones,
    "dataset_levenshtein_filtro_1_evaluaciones": dataset_levenshtein_filtro_1_evaluaciones,
    "dataset_overlap_filtro_1_evaluaciones": dataset_overlap_filtro_1_evaluaciones,
    "dataset_sbert_filtro_1_evaluaciones": dataset_sbert_filtro_1_evaluaciones,
    "dataset_web_similitary_filtro_1_evaluaciones": dataset_web_similitary_filtro_1_evaluaciones,
    "dataset_pixel_to_pixel_filtro_1_evaluaciones": dataset_pixel_to_pixel_filtro_1_evaluaciones}

diccionario_dataset_filtro_2_para_porcentajes_de_mejoras = {
    "dataset_bag_filtro_2_evaluaciones": dataset_bag_filtro_2_evaluaciones,
    "dataset_cosine_filtro_2_evaluaciones": dataset_cosine_filtro_2_evaluaciones,
    "dataset_jaccard_filtro_2_evaluaciones": dataset_jaccard_filtro_2_evaluaciones,
    "dataset_levenshtein_filtro_2_evaluaciones": dataset_levenshtein_filtro_2_evaluaciones,
    "dataset_overlap_filtro_2_evaluaciones": dataset_overlap_filtro_2_evaluaciones,
    "dataset_sbert_filtro_2_evaluaciones": dataset_sbert_filtro_2_evaluaciones,
    "dataset_web_similitary_filtro_2_evaluaciones": dataset_web_similitary_filtro_2_evaluaciones,
    "dataset_pixel_to_pixel_filtro_2_evaluaciones": dataset_pixel_to_pixel_filtro_2_evaluaciones}


def porcentaje_mejoras_filtro_1(dataset_baseline):
    # Hacer una copia del DataFrame original
    dataset_base = dataset_baseline.copy()
    
    # Crear un diccionario para almacenar los resultados
    diccionario_pixel_to_pixel_mejoras_filtro_1 = {}
    
    # Iterar sobre las claves en lista_claves
    for x in diccionario_dataset_filtro_1_para_porcentajes_de_mejoras.keys():

        #Utilizamos el metodo assing porque de forma clasica no asigna el valor
        # calculamos porcentaje de mejora k1
        dataset_base = dataset_base.assign(Porcentaje_de_mejora_K1=((diccionario_dataset_filtro_1_para_porcentajes_de_mejoras[x]["promedio_1"]- dataset_base["promedio_1"] )/ dataset_base["promedio_1"])*100)

        # calculamos porcentaje de mejora k2
        dataset_base = dataset_base.assign(Porcentaje_de_mejora_K2=((diccionario_dataset_filtro_1_para_porcentajes_de_mejoras[x]["promedio_2"]- dataset_base["promedio_2"] )/ dataset_base["promedio_2"])*100)

        # calculamos porcentaje de mejora k3
        dataset_base = dataset_base.assign(Porcentaje_de_mejora_K3=((diccionario_dataset_filtro_1_para_porcentajes_de_mejoras[x]["promedio_3"]- dataset_base["promedio_3"] )/ dataset_base["promedio_3"])*100)

        
        # Guardar el resultado en el diccionario
        diccionario_pixel_to_pixel_mejoras_filtro_1[f"dataset_pixel_mejora_filtro_1_{x}"] = dataset_base
    
    return diccionario_pixel_to_pixel_mejoras_filtro_1



def porcentaje_mejoras_filtro_2(dataset_baseline):
    # Hacer una copia del DataFrame original
    dataset_base = dataset_baseline.copy()
    
    # Crear un diccionario para almacenar los resultados
    diccionario_pixel_to_pixel_mejoras_filtro_2 = {}
    
    # Iterar sobre las claves en lista_claves
    for x in diccionario_dataset_filtro_2_para_porcentajes_de_mejoras.keys():

        #Utilizamos el metodo assing porque de forma clasica no asigna el valor
        # calculamos porcentaje de mejora k1
        dataset_base = dataset_base.assign(Porcentaje_de_mejora_K1=((diccionario_dataset_filtro_2_para_porcentajes_de_mejoras[x]["promedio_1"]- dataset_base["promedio_1"] )/ dataset_base["promedio_1"])*100)

        # calculamos porcentaje de mejora k2
        dataset_base = dataset_base.assign(Porcentaje_de_mejora_K2=((diccionario_dataset_filtro_2_para_porcentajes_de_mejoras[x]["promedio_2"]- dataset_base["promedio_2"] )/ dataset_base["promedio_2"])*100)

        # calculamos porcentaje de mejora k3
        dataset_base = dataset_base.assign(Porcentaje_de_mejora_K3=((diccionario_dataset_filtro_2_para_porcentajes_de_mejoras[x]["promedio_3"]- dataset_base["promedio_3"] )/ dataset_base["promedio_3"])*100)

        
        # Guardar el resultado en el diccionario
        diccionario_pixel_to_pixel_mejoras_filtro_2[f"dataset_pixel_mejora_filtro_2_{x}"] = dataset_base
    
    return diccionario_pixel_to_pixel_mejoras_filtro_2
        

# Porcentajes de mejoras

In [69]:
# Porcentajes de mejora de filtro 1
diccionario_porcentajes_mejoras_filtro_1=porcentaje_mejoras_filtro_1(diccionario_dataset_filtro_1_para_porcentajes_de_mejoras["dataset_pixel_to_pixel_filtro_1_evaluaciones"])

In [70]:
#Porcentajes de mejora de filtro 2
diccionario_porcentajes_mejoras_filtro_2=porcentaje_mejoras_filtro_2(diccionario_dataset_filtro_2_para_porcentajes_de_mejoras["dataset_pixel_to_pixel_filtro_2_evaluaciones"])

## Tabla de comparativas

In [71]:
#Valores para K1
Promedio_porcentaje_mejora_pixel_v_bag_K1_filtro_1=diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_bag_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_cosine_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_cosine_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_jaccard_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_jaccard_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_levestehinh_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_levenshtein_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_overlap_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_overlap_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_sbert_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_sbert_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_web_similitary_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_web_similitary_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_pixel_K1_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_pixel_to_pixel_filtro_1_evaluaciones"].Porcentaje_de_mejora_K1.mean()

Promedio_porcentaje_mejora_pixel_v_bag_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_bag_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_cosine_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_cosine_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_jaccard_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_jaccard_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_levestehinh_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_levenshtein_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_overlap_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_overlap_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_sbert_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_sbert_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_web_similitary_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_web_similitary_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()
Promedio_porcentaje_mejora_pixel_v_pixel_K1_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_pixel_to_pixel_filtro_2_evaluaciones"].Porcentaje_de_mejora_K1.mean()



Valores_k1=[

    Promedio_porcentaje_mejora_pixel_v_bag_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_cosine_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_jaccard_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_levestehinh_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_overlap_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_sbert_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_web_similitary_K1_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_pixel_K1_filtro_1,

    Promedio_porcentaje_mejora_pixel_v_bag_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_cosine_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_jaccard_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_levestehinh_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_overlap_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_sbert_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_web_similitary_K1_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_pixel_K1_filtro_2
]



#Valores para k2
Promedio_porcentaje_mejora_pixel_v_bag_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_bag_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_cosine_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_cosine_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_jaccard_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_jaccard_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_levestehinh_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_levenshtein_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_overlap_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_overlap_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_sbert_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_sbert_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_web_similitary_K2_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_web_similitary_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_pixel_K2_filtro_1 = diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_pixel_to_pixel_filtro_1_evaluaciones"].Porcentaje_de_mejora_K2.mean()



Promedio_porcentaje_mejora_pixel_v_bag_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_bag_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_cosine_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_cosine_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_jaccard_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_jaccard_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_levestehinh_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_levenshtein_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_overlap_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_overlap_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_sbert_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_sbert_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_web_similitary_K2_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_web_similitary_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()
Promedio_porcentaje_mejora_pixel_v_pixel_K2_filtro_2 = diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_pixel_to_pixel_filtro_2_evaluaciones"].Porcentaje_de_mejora_K2.mean()

Valores_k2=[

    Promedio_porcentaje_mejora_pixel_v_bag_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_cosine_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_jaccard_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_levestehinh_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_overlap_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_sbert_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_web_similitary_K2_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_pixel_K2_filtro_1,

    Promedio_porcentaje_mejora_pixel_v_bag_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_cosine_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_jaccard_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_levestehinh_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_overlap_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_sbert_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_web_similitary_K2_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_pixel_K2_filtro_2
]


#Valores para k3
Promedio_porcentaje_mejora_pixel_v_bag_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_bag_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_cosine_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_cosine_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_jaccard_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_jaccard_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_levestehinh_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_levenshtein_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_overlap_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_overlap_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_sbert_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_sbert_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_web_similitary_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_web_similitary_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_pixel_K3_filtro_1= diccionario_porcentajes_mejoras_filtro_1["dataset_pixel_mejora_filtro_1_dataset_pixel_to_pixel_filtro_1_evaluaciones"].Porcentaje_de_mejora_K3.mean()



Promedio_porcentaje_mejora_pixel_v_bag_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_bag_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_cosine_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_cosine_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_jaccard_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_jaccard_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_levestehinh_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_levenshtein_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_overlap_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_overlap_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_sbert_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_sbert_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_web_similitary_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_web_similitary_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()
Promedio_porcentaje_mejora_pixel_v_pixel_K3_filtro_2= diccionario_porcentajes_mejoras_filtro_2["dataset_pixel_mejora_filtro_2_dataset_pixel_to_pixel_filtro_2_evaluaciones"].Porcentaje_de_mejora_K3.mean()



Valores_k3=[

    Promedio_porcentaje_mejora_pixel_v_bag_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_cosine_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_jaccard_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_levestehinh_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_overlap_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_sbert_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_web_similitary_K3_filtro_1,
    Promedio_porcentaje_mejora_pixel_v_pixel_K3_filtro_1,

    Promedio_porcentaje_mejora_pixel_v_bag_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_cosine_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_jaccard_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_levestehinh_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_overlap_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_sbert_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_web_similitary_K3_filtro_2,
    Promedio_porcentaje_mejora_pixel_v_pixel_K3_filtro_2
]

In [72]:
Tabla_comparativa= {
    'Metrica':["Bag", "Cosine", "jaccard", "levestehin","overlap", "SBERT", "Web_similitary","pixel_to_pixel", "Bag", "Cosine", "jaccard", "levestehin","overlap", "SBERT", "Web_similitary","pixel_to_pixel"],
    'Filtro':["1","1","1","1","1","1","1","1","2","2","2","2","2","2","2","2"],
    'K1':Valores_k1,
    'K2':Valores_k2,
    'k3':Valores_k3
}

Tabla_comparativa=pd.DataFrame(Tabla_comparativa)

In [78]:
Tabla_comparativa.to_csv("Tabla_comparativa_ROAD.csv")

In [73]:
Tabla_comparativa

,Metrica,Filtro,K1,K2,k3
0,Bag,1,34.507012,5.896292,1.222675
1,Cosine,1,37.152807,6.195170,2.025761
2,jaccard,1,37.466116,6.032266,1.914447
3,levestehin,1,43.921700,10.099348,4.143133
4,overlap,1,49.898480,10.343039,5.147501
5,SBERT,1,42.803367,8.969893,3.843307
6,Web_similitary,1,42.012970,8.280656,4.647421
7,pixel_to_pixel,1,0.000000,0.000000,0.000000
8,Bag,2,13.958270,-2.972509,-4.029065
9,Cosine,2,18.595909,-0.178131,-1.776467


In [40]:
Tabla_comparativa

,Metrica,Filtro,K1,K2,k3
0,Bag,1,34.507012,5.896292,1.222675
1,Cosine,1,37.152807,6.195170,2.025761
2,jaccard,1,37.466116,6.032266,1.914447
3,levestehin,1,43.921700,10.099348,4.143133
4,overlap,1,49.898480,10.343039,5.147501
5,SBERT,1,42.803367,8.969893,3.843307
6,Web_similitary,1,42.012970,8.280656,4.647421
7,pixel_to_pixel,1,0.000000,0.000000,0.000000
8,Bag,2,-13.958270,2.972509,4.029065
9,Cosine,2,-18.595909,0.178131,1.776467


In [57]:
Tabla_comparativa

,Metrica,Filtro,K1,K2,k3
0,Bag,1,6.598042,3.817290,1.903762
1,Cosine,1,9.518240,7.710590,5.751998
2,jaccard,1,9.032808,7.777194,5.893264
3,levestehin,1,8.203453,6.217435,4.531543
4,overlap,1,11.487525,9.166865,6.430232
5,SBERT,1,10.008185,8.766537,6.196489
6,Web_similitary,1,11.284157,8.892075,6.027652
7,pixel_to_pixel,1,0.000000,0.000000,0.000000
8,Bag,2,-4.091452,-4.248340,-3.987095
9,Cosine,2,2.757261,1.244034,1.602530


## Estudio negativos 

- Porcentaje de casos negativos con respecto al total


In [74]:
def porcentaje_de_casos_negativos_respecto_total_k():
    
    valores_para_porcentaje=[]

    for k in range(1,4):
    # Iterar sobre las claves en lista_claves
        for x in diccionario_porcentajes_mejoras_filtro_1.keys():
            valores_totales=len(diccionario_porcentajes_mejoras_filtro_1[x])
            valores_negativos= len(diccionario_porcentajes_mejoras_filtro_1[x][diccionario_porcentajes_mejoras_filtro_1[x].loc[:,f"Porcentaje_de_mejora_K{k}"]<0])
            valores_para_porcentaje.extend([(valores_totales,valores_negativos,x, "filtro_1", k, "bag" if "bag" in x else ("cosine" if "cosine" in x else ("jaccard" if "jaccard" in x else ("levestehin" if "levestehing" in x else 
                                                                                                                                                                                             ("overlap" if "overlap" in x else
                                                                                                                                                                                              ("sbert" if "sbert" in x else 
                                                                                                                                                                                               ("web" if "web" in x else "pixel")))))))])

        for x in diccionario_porcentajes_mejoras_filtro_2.keys():
            valores_totales=len(diccionario_porcentajes_mejoras_filtro_2[x])
            valores_negativos= len(diccionario_porcentajes_mejoras_filtro_2[x][diccionario_porcentajes_mejoras_filtro_2[x].loc[:,f"Porcentaje_de_mejora_K{k}"]<0])
            valores_para_porcentaje.extend([(valores_totales,valores_negativos,x, "filtro_2", k, "bag" if "bag" in x else ("cosine" if "cosine" in x else ("jaccard" if "jaccard" in x else ("levestehin" if "levestehing" in x else 
                                                                                                                                                                                             ("overlap" if "overlap" in x else
                                                                                                                                                                                              ("sbert" if "sbert" in x else 
                                                                                                                                                                                               ("web" if "web" in x else "pixel")))))))])

    return valores_para_porcentaje

valores=porcentaje_de_casos_negativos_respecto_total_k()

In [75]:
data = pd.DataFrame(columns=["Valores_totales","Valores_negativos","dataset","filtro","K", "Metrica"], data=valores)

In [ ]:
-Porcenajte con respecto al total 
-Porcentaje de casos negativos en los que se utiliza un filtro u otro
- Porcentaje de casos negativos en los que se utiliza una métrica de similitud u otro
- Media de valores de los porcentajes
- Valor minimo del porcentaje
- Valor maxímo del porcentaje

In [79]:
data.to_csv("Estudio_negativos.csv")

## Para formulario

In [77]:

dataset_animal_respuestas_completo_bytes_reparado.to_pickle("dataset_animal_completo_info_respuesta_for_3.pkl")

# Para consultas

## query indice 0 (tomada al azar)

In [38]:
funcion_k(diccionario_datasets_filtro_1["salida_animal_overlap"],0,3,"salida_animal_overlap")

,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
6,6,0-529,0.961331,0.000090,0,529,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,"[136, 147, 127, 124, 123, 137, 127, 111, 116, ...",[0.38248765],[0.4036259],"[(529, 0.4036259055137634)]",0.403626,"[(529, 0.4036259055137634), (278, 0.4372065365...",0.420416,"[(529, 0.4036259055137634), (278, 0.4372065365...",0.415254
3,3,0-278,0.959185,0.000084,0,278,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,"[85, 85, 73, 72, 89, 78, 92, 101, 135, 109, 82...",[0.38248765],[0.43720654],"[(529, 0.4036259055137634)]",0.403626,"[(529, 0.4036259055137634), (278, 0.4372065365...",0.420416,"[(529, 0.4036259055137634), (278, 0.4372065365...",0.415254
9,9,0-762,0.958351,0.000082,0,762,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,"[92, 92, 137, 147, 87, 74, 74, 54, 82, 116, 96...",[0.38248765],[0.40493098],"[(529, 0.4036259055137634)]",0.403626,"[(529, 0.4036259055137634), (278, 0.4372065365...",0.420416,"[(529, 0.4036259055137634), (278, 0.4372065365...",0.415254


In [39]:
funcion_k(diccionario_datasets_filtro_1["salida_animal_pixel_to_pixel"],0,3,"salida_animal_pixel_to_pixel")

,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
9,9,0-762,87.385091,0.462068,0,762,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,"[92, 92, 137, 147, 87, 74, 74, 54, 82, 116, 96...",[0.38248765],[0.40493098],"[(762, 0.404930979013443)]",0.404931,"[(762, 0.404930979013443), (870, 0.36364752054...",0.384289,"[(762, 0.404930979013443), (870, 0.36364752054...",0.380117
11,11,0-870,81.966797,0.421995,0,870,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,"[91, 132, 169, 170, 151, 103, 97, 116, 146, 16...",[0.38248765],[0.36364752],"[(762, 0.404930979013443)]",0.404931,"[(762, 0.404930979013443), (870, 0.36364752054...",0.384289,"[(762, 0.404930979013443), (870, 0.36364752054...",0.380117
10,10,0-818,75.324870,0.372872,0,818,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,"[39, 47, 61, 62, 76, 69, 50, 40, 56, 146, 168,...",[0.38248765],[0.37177283],"[(762, 0.404930979013443)]",0.404931,"[(762, 0.404930979013443), (870, 0.36364752054...",0.384289,"[(762, 0.404930979013443), (870, 0.36364752054...",0.380117


## query indice 316 (tomada al azar)

In [40]:
funcion_k(diccionario_datasets_filtro_1["salida_animal_overlap"],316,3,"salida_animal_overlap")


,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
680,680,316-85,351.750901,0.999963,316,85,"[73, 91, 71, 111, 63, 45, 40, 42, 42, 50, 46, ...",leopard,"[53, 46, 65, 107, 99, 53, 46, 45, 44, 45, 43, ...",[0.36382127],[0.41753176],"[(85, 0.4175317585468292)]",0.417532,"[(85, 0.4175317585468292), (338, 0.36778670549...",0.392659,"[(85, 0.4175317585468292), (338, 0.36778670549...",0.379178
674,674,316-338,351.749540,0.999960,316,338,"[73, 91, 71, 111, 63, 45, 40, 42, 42, 50, 46, ...",leopard,"[53, 56, 58, 58, 57, 59, 64, 65, 62, 62, 63, 6...",[0.36382127],[0.3677867],"[(85, 0.4175317585468292)]",0.417532,"[(85, 0.4175317585468292), (338, 0.36778670549...",0.392659,"[(85, 0.4175317585468292), (338, 0.36778670549...",0.379178
679,679,316-812,351.747454,0.999954,316,812,"[73, 91, 71, 111, 63, 45, 40, 42, 42, 50, 46, ...",leopard,"[162, 171, 193, 184, 180, 184, 172, 144, 140, ...",[0.36382127],[0.3522157],"[(85, 0.4175317585468292)]",0.417532,"[(85, 0.4175317585468292), (338, 0.36778670549...",0.392659,"[(85, 0.4175317585468292), (338, 0.36778670549...",0.379178


In [41]:
funcion_k(diccionario_datasets_filtro_1["salida_animal_pixel_to_pixel"],316,3,"salida_animal_pixel_to_pixel")

,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
673,673,316-28,108.166016,0.615762,316,28,"[73, 91, 71, 111, 63, 45, 40, 42, 42, 50, 46, ...",leopard,"[255, 252, 252, 252, 252, 234, 192, 182, 221, ...",[0.36382127],[0.36457676],"[(28, 0.3645767569541931)]",0.364577,"[(28, 0.3645767569541931), (250, 0.37088230252...",0.36773,"[(28, 0.3645767569541931), (250, 0.37088230252...",0.364983
672,672,316-250,73.618490,0.360252,316,250,"[73, 91, 71, 111, 63, 45, 40, 42, 42, 50, 46, ...",leopard,"[109, 117, 115, 113, 120, 127, 131, 133, 146, ...",[0.36382127],[0.3708823],"[(28, 0.3645767569541931)]",0.364577,"[(28, 0.3645767569541931), (250, 0.37088230252...",0.36773,"[(28, 0.3645767569541931), (250, 0.37088230252...",0.364983
675,675,316-523,67.728516,0.316690,316,523,"[73, 91, 71, 111, 63, 45, 40, 42, 42, 50, 46, ...",leopard,"[228, 230, 236, 232, 239, 239, 234, 232, 238, ...",[0.36382127],[0.35949117],"[(28, 0.3645767569541931)]",0.364577,"[(28, 0.3645767569541931), (250, 0.37088230252...",0.36773,"[(28, 0.3645767569541931), (250, 0.37088230252...",0.364983


## query indice 368 (tomada al azar)

In [42]:
funcion_k(diccionario_datasets_filtro_2["salida_animal_overlap"],368,3,"salida_animal_overlap")


,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
2636,2636,368-278,351.736133,0.999962,368,278,"[83, 106, 125, 121, 150, 177, 181, 173, 159, 1...",camel,"[85, 85, 73, 72, 89, 78, 92, 101, 135, 109, 82...",[0.35193616],[0.43720654],"[(278, 0.43720653653144836)]",0.437207,"[(278, 0.43720653653144836), (686, 0.436853557...",0.43703,"[(278, 0.43720653653144836), (686, 0.436853557...",0.415278
2649,2649,368-686,351.735348,0.999960,368,686,"[83, 106, 125, 121, 150, 177, 181, 173, 159, 1...",camel,"[255, 254, 255, 255, 255, 255, 255, 255, 255, ...",[0.35193616],[0.43685356],"[(278, 0.43720653653144836)]",0.437207,"[(278, 0.43720653653144836), (686, 0.436853557...",0.43703,"[(278, 0.43720653653144836), (686, 0.436853557...",0.415278
2654,2654,368-818,351.735099,0.999959,368,818,"[83, 106, 125, 121, 150, 177, 181, 173, 159, 1...",camel,"[39, 47, 61, 62, 76, 69, 50, 40, 56, 146, 168,...",[0.35193616],[0.37177283],"[(278, 0.43720653653144836)]",0.437207,"[(278, 0.43720653653144836), (686, 0.436853557...",0.43703,"[(278, 0.43720653653144836), (686, 0.436853557...",0.415278


In [43]:
funcion_k(diccionario_datasets_filtro_2["salida_animal_pixel_to_pixel"],368,3,"salida_animal_pixel_to_pixel")

,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
2629,2629,368-1,91.112956,0.408978,368,1,"[83, 106, 125, 121, 150, 177, 181, 173, 159, 1...",camel,"[255, 253, 253, 253, 253, 253, 253, 253, 253, ...",[0.35193616],[0.410904],"[(1, 0.4109039902687073)]",0.410904,"[(1, 0.4109039902687073), (238, 0.419674932956...",0.415289,"[(1, 0.4109039902687073), (238, 0.419674932956...",0.390816
2634,2634,368-238,91.090169,0.408834,368,238,"[83, 106, 125, 121, 150, 177, 181, 173, 159, 1...",camel,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[0.35193616],[0.41967493],"[(1, 0.4109039902687073)]",0.410904,"[(1, 0.4109039902687073), (238, 0.419674932956...",0.415289,"[(1, 0.4109039902687073), (238, 0.419674932956...",0.390816
2647,2647,368-677,84.490560,0.367258,368,677,"[83, 106, 125, 121, 150, 177, 181, 173, 159, 1...",camel,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[0.35193616],[0.34186885],"[(1, 0.4109039902687073)]",0.410904,"[(1, 0.4109039902687073), (238, 0.419674932956...",0.415289,"[(1, 0.4109039902687073), (238, 0.419674932956...",0.390816


## query indice 732 (tomada al azar)

In [44]:
funcion_k(diccionario_datasets_filtro_2["salida_animal_overlap"],732,3,"salida_animal_overlap")


,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
5652,5652,732-711,351.721205,0.999920,732,711,"[49, 39, 49, 70, 66, 45, 46, 60, 55, 44, 97, 1...",aquarium_fish,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[0.36735365],[0.40704018],"[(711, 0.40704017877578735)]",0.40704,"[(711, 0.40704017877578735), (577, 0.394565194...",0.400803,"[(711, 0.40704017877578735), (577, 0.394565194...",0.40417
5646,5646,732-577,351.716358,0.999906,732,577,"[49, 39, 49, 70, 66, 45, 46, 60, 55, 44, 97, 1...",aquarium_fish,"[105, 111, 135, 175, 188, 208, 210, 190, 209, ...",[0.36735365],[0.3945652],"[(711, 0.40704017877578735)]",0.40704,"[(711, 0.40704017877578735), (577, 0.394565194...",0.400803,"[(711, 0.40704017877578735), (577, 0.394565194...",0.40417
5629,5629,732-1,351.716083,0.999905,732,1,"[49, 39, 49, 70, 66, 45, 46, 60, 55, 44, 97, 1...",aquarium_fish,"[255, 253, 253, 253, 253, 253, 253, 253, 253, ...",[0.36735365],[0.410904],"[(711, 0.40704017877578735)]",0.40704,"[(711, 0.40704017877578735), (577, 0.394565194...",0.400803,"[(711, 0.40704017877578735), (577, 0.394565194...",0.40417


In [45]:
funcion_k(diccionario_datasets_filtro_2["salida_animal_pixel_to_pixel"],732,3,"salida_animal_pixel_to_pixel")

,Unnamed: 0,PAR_DE_INDICE,AVERAGE_METRICA,AVERAGE_METRICA_NORMALIZE,PAR_1,PAR_2,data_PAR_1,etiqueta,data_PAR_2,ROAD_PAR_1,ROAD_PAR_2,Evaluacion_k_igual1,promedio_1,Evaluacion_k_igual2,promedio_2,Evaluacion_k_igual3,promedio_3
5648,5648,732-677,139.801758,0.715707,732,677,"[49, 39, 49, 70, 66, 45, 46, 60, 55, 44, 97, 1...",aquarium_fish,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[0.36735365],[0.34186885],"[(677, 0.3418688476085663)]",0.341869,"[(677, 0.3418688476085663), (1, 0.410903990268...",0.376386,"[(677, 0.3418688476085663), (1, 0.410903990268...",0.390816
5629,5629,732-1,137.557617,0.701570,732,1,"[49, 39, 49, 70, 66, 45, 46, 60, 55, 44, 97, 1...",aquarium_fish,"[255, 253, 253, 253, 253, 253, 253, 253, 253, ...",[0.36735365],[0.410904],"[(677, 0.3418688476085663)]",0.341869,"[(677, 0.3418688476085663), (1, 0.410903990268...",0.376386,"[(677, 0.3418688476085663), (1, 0.410903990268...",0.390816
5635,5635,732-238,134.812826,0.684278,732,238,"[49, 39, 49, 70, 66, 45, 46, 60, 55, 44, 97, 1...",aquarium_fish,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[0.36735365],[0.41967493],"[(677, 0.3418688476085663)]",0.341869,"[(677, 0.3418688476085663), (1, 0.410903990268...",0.376386,"[(677, 0.3418688476085663), (1, 0.410903990268...",0.390816
